In [827]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [828]:
df = pd.read_csv('responses2.csv')

In [829]:
print('List of non-integer columns:')
for id,dtyp in df.dtypes.iteritems():
    if dtyp=='object':
        print('-',id)

List of non-integer columns:
- Smoking
- Alcohol
- Punctuality
- Lying
- Internet usage
- Gender
- Left - right handed
- Education
- Only child
- Village - town
- House - block of flats


In [830]:
#for id,val in df.isnull().sum().iteritems():
#    if val>(0.2 * len(df)):
#        print(id,val)
#No columns with huge number of missing values

#print("Columns with largest missing value count")
#for id,val in df.isnull().sum().iteritems():
#    if val>(0.01 * len(df)):
#        print(id,'\t\t',val)
        
for col in list(df):
    df[col].fillna(df[col].mode()[0], inplace = True)  
    
for id,val in df.isnull().sum().iteritems():
    if val>0:
        print(id,val)

In [831]:
le = LabelEncoder()
categoricalFeatures = []
for col in [id for id,dtyp in df.dtypes.iteritems() if dtyp=='object']:
    categoricalFeatures.append(col)
    df[col] = le.fit_transform(df[col].astype('str'))
    #print(col,"\n",df[col][:5])
#df = df.drop(columns=categoricalFeatures)

In [832]:
df2 = df[categoricalFeatures]

In [833]:
print("DF shape: ",df.shape)
print("DF2 shape: ",df2.shape)

DF shape:  (1010, 150)
DF2 shape:  (1010, 11)


In [834]:
df=df.drop(categoricalFeatures, axis=1)

In [835]:
ohe = OneHotEncoder()
#for col in categoricalFeatures:
df = df.join(pd.DataFrame(ohe.fit_transform(df2).toarray()))
print("DF shape: ",df.shape)

DF shape:  (1010, 173)


In [836]:
emp = df['Empathy']
df = df.drop('Empathy',axis=1)

In [824]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
rescaledX = MinMaxScaler(feature_range=(0, 1)).fit_transform(df)
standardizedX = StandardScaler().fit_transform(rescaledX)
normalizedX = Normalizer().fit_transform(standardizedX)

In [705]:
from sklearn.preprocessing import Normalizer
newdf = Normalizer().fit_transform(df)

In [780]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score

dtree = DecisionTreeClassifier(max_depth=2)
knn = KNeighborsClassifier()
#svc = SVC(kernel='poly',degree=2, gamma = 1.2, C=1.2, coef0=1)
svc = LinearSVC(dual=False)#dual=False, C=2)
gnb = GaussianNB()
mlp = MLPClassifier()

In [ ]:
t = list(df)
t.remove('Empathy')

In [ ]:
emp.shape

In [837]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, emp, test_size=0.2,random_state = 0)#(df[t], df['Empathy'], test_size=0.2)

In [838]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2,random_state = 0)

In [839]:
gnb.fit(X_train,y_train)
print(gnb.score(X_dev,y_dev))

0.376543209877


In [710]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.01, 0.1, 2, 4, 8, 10], 'degree':[1,2,3,4,5,6]}
#parameters = {'dual':(False, True), 'C':[0.001, 0.01, 0.1, 1, 2, 3, 4, 5, 10]}
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)
clf.score(X_dev, y_dev)

0.40740740740740738

In [699]:
clf.best_params_

{'C': 2, 'degree': 1}

In [840]:
accSum = []
#for i in range(10):
svc.fit(X_train,y_train)
#accSum.append(
svc.score(X_dev,y_dev)#)    
#np.mean(accSum)

0.3888888888888889

In [756]:
accSum

[]

In [841]:
accSum = []
#for i in range(10):
dtree.fit(X_train,y_train)
#accSum.append(
dtree.score(X_dev,y_dev)#)
#np.mean(accSum)

0.39506172839506171

In [781]:
accSum = []
for i in range(10):
    mlp.fit(X_train,y_train)
    accSum.append(mlp.score(X_dev,y_dev))
print("Dev: ",np.mean(accSum))
print("Test: ",mlp.score(X_test,y_test))

C:\Users\chinm\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Dev:  0.393209876543
Test:  0.356435643564


In [843]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=9,max_depth=6, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_dev, y_dev)

0.3888888888888889

In [813]:
acc = 0
for nest in range(1,13):
    for dep in range(5,11):
        rfc = RandomForestClassifier(n_estimators=nest,max_depth=dep, random_state=0)
        rfc.fit(X_train, y_train)
        devScore = rfc.score(X_dev, y_dev)
        if devScore > acc:
            acc = devScore
            print()
            print(nest,dep)
            print("RandomForest score on dev",rfc.score(X_dev, y_dev))



1 5
RandomForest score on dev 0.333333333333

1 7
RandomForest score on dev 0.37037037037

2 5
RandomForest score on dev 0.376543209877

3 6
RandomForest score on dev 0.395061728395

5 5
RandomForest score on dev 0.401234567901

9 6
RandomForest score on dev 0.413580246914

11 7
RandomForest score on dev 0.425925925926


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

accList=[]
bestAcc = 0.1
bestK = 1
for i in range(5):
    for k in range(1,170,10):
        
        X_new = SelectKBest(chi2, k).fit_transform(X_dev, y_dev)
        svcTuned.fit()
        
        #nowAcc = np.mean(cross_val_score(svc, X_new, df['Empathy'],cv=10))
        if nowAcc>bestAcc:
            bestAcc = nowAcc
            #bestK = k
    accList.append(bestAcc)
    print(bestAcc)
        
#print('k=',bestK)
print('Accuracy of baseline=',np.mean(accList))

In [648]:
kbesht = SelectKBest(chi2,20)
X_new = kbesht.fit_transform(X_dev, y_dev)
relevantCols = []
for col in kbesht.get_support(indices=True):
    relevantCols.append(df.columns[col])

#print(relevantCols)    

#X_train[relevantCols]
X_train = X_train[relevantCols]
X_dev = X_dev[relevantCols]
X_test = X_test[relevantCols]

In [845]:
X_new = SelectKBest(chi2, 11).fit_transform(X_train, y_train)
relevantCols = []
for col in kbesht.get_support(indices=True):
    relevantCols.append(df.columns[col])
rfc = RandomForestClassifier(n_estimators=10,max_depth=9, random_state=0)
rfc.fit(X_train[relevantCols], y_train)
print("RandomForest score on dev",rfc.score(X_dev[relevantCols], y_dev))
print("RandomForest score on test",rfc.score(X_test[relevantCols], y_test))

RandomForest score on dev 0.376543209877
RandomForest score on test 0.371287128713


In [600]:
svcTuned = LinearSVC(dual=False)
accSum = []
for i in range(10):
    svcTuned.fit(X_train,y_train)
    accSum.append(svcTuned.score(X_dev,y_dev))    
print("Dev: ",np.mean(accSum))
print("Test: ",svcTuned.score(X_test, y_test))

Dev:  0.382716049383
Test:  0.366336633663


In [603]:
dtreeTuned = DecisionTreeClassifier(max_depth=2)
accSum = []
for i in range(10):
    dtreeTuned.fit(X_train,y_train)
    accSum.append(dtreeTuned.score(X_dev,y_dev))
print("Dev: ",np.mean(accSum))
print("Test: ",dtreeTuned.score(X_test,y_test))

Dev:  0.444444444444
Test:  0.356435643564
